<div class="alert alert-block alert-success">

# **1.** Environment Setup

<div>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu May  1 00:02:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


### 1.1 Connect Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os

# Change to the directory where project is located
os.chdir('/content/drive/MyDrive/FACULDADE/Deep Learning/project')
# os.chdir('/content/drive/MyDrive/College/MSc/2nd Semester/Deep Learning/project')

# Verify that we changed the directory
print("Changed directory to:", os.getcwd())

Changed directory to: /content/drive/.shortcut-targets-by-id/1hNB4s6RR7JeKfFdGwm27WrF6pNXXIg3R/Deep Learning/project


## 1.2 Import Libraries

In [5]:
# Google Colab
!pip install keras_cv

In [6]:
import pandas as pd
import zipfile
import pickle
from sklearn.preprocessing import LabelEncoder
from pathlib import Path
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import regularizers
from classes import *
from functions import *

In [7]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Dropout, Input, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.metrics import AUC, F1Score, CategoricalAccuracy, TopKCategoricalAccuracy
from sklearn.metrics import f1_score, precision_score, recall_score
from keras import optimizers

## 1.3 Import Dataset

The amount of data we have is not supported by GitHub (where we have our project stored). The solution is: create a folder named data and allocate the rare_species file inside it. The gitignore file makes sure this folder is not used when we are pulling or pushing changes but everyone needs to have it on their machines locally. A random seed was used to ensure that the splits stay the same.

In [8]:
# Import Metadata
metadata_path = Path("../data/rare_species/metadata.csv")
df = pd.read_csv(metadata_path)
df.head()

,rare_species_id,eol_content_id,eol_page_id,kingdom,phylum,family,file_path
0,75fd91cb-2881-41cd-88e6-de451e8b60e2,12853737,449393,animalia,mollusca,unionidae,mollusca_unionidae/12853737_449393_eol-full-si...
1,28c508bc-63ff-4e60-9c8f-1934367e1528,20969394,793083,animalia,chordata,geoemydidae,chordata_geoemydidae/20969394_793083_eol-full-...
2,00372441-588c-4af8-9665-29bee20822c0,28895411,319982,animalia,chordata,cryptobranchidae,chordata_cryptobranchidae/28895411_319982_eol-...
3,29cc6040-6af2-49ee-86ec-ab7d89793828,29658536,45510188,animalia,chordata,turdidae,chordata_turdidae/29658536_45510188_eol-full-s...
4,94004bff-3a33-4758-8125-bf72e6e57eab,21252576,7250886,animalia,chordata,indriidae,chordata_indriidae/21252576_7250886_eol-full-s...


In [9]:
df.shape # 11983 images

(11983, 7)

<div class="alert alert-block alert-success">

# **2.** Preprocessing

<div>

In [10]:
#Load the DataFrames from the .pkl files
with open("../data/train_df.pkl", "rb") as f:
     train_df = pickle.load(f)

with open("../data/valid_df.pkl", "rb") as f:
     val_df = pickle.load(f)

with open("../data/test_df.pkl", "rb") as f:
     test_df = pickle.load(f)

with open("family_encoder.pkl", "rb") as f:
     family_encoder = pickle.load(f)

with open("phylum_encoder.pkl", "rb") as f:
     phylum_encoder = pickle.load(f)

In [11]:
# identify the minority class
minority_class = train_df['family'].value_counts()[train_df['family'].value_counts() < 25].index
minority_class = minority_class.to_list()

In [12]:
batch_size = 32 ## the less the better because in each epoch the model sees N / batch_size images
image_size = (224, 224)

preprocess = Preprocessor_with_phylum(image_size=image_size, batch_size=batch_size)

In [13]:
# num_images = 16 ##
# rows, cols = 4, 4 ##

# plot_batch(train_ds, class_names=class_names, num_images=num_images, rows=rows, cols=cols)

<div class="alert alert-block alert-success">

# **3.** Models

<div>

## EfficientNet

### Base line 1 (without preprocessing and without regularization)

#### Set-up

In [ ]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input

In [ ]:
# Compile with metrics
verbose = 1
metrics = [
    CategoricalAccuracy(name="accuracy"),
    AUC(name="auc"),
    F1Score(average="macro", name="f1_macro"),
    F1Score(average="weighted", name="f1_weighted"),
    TopKCategoricalAccuracy(k=5, name="top5_accuracy")
]

In [ ]:
# Load datasets
train_ds_en_no_proc_no_reg, family_class_names, phylum_class_names = preprocess.load_img(
    train_df,
    minority_class=[],
    augment=None,
    oversampling=False,
    shuffle=True,
    preprocessing_function=preprocess_input,
    family_encoder=family_encoder,
    phylum_encoder=phylum_encoder,
)

val_ds_en_no_proc_no_reg, _, _ = preprocess.load_img(
    val_df,
    minority_class=[],
    augment=None,
    oversampling=False,
    shuffle=False,
    preprocessing_function=preprocess_input,
    family_encoder=family_encoder,
    phylum_encoder=phylum_encoder,
)

#### Run

In [ ]:
# Image input pipeline
image_input = Input(shape=(224, 224, 3), name="image_input")  # Input for RGB image
base_model = EfficientNetB0(include_top=False, weights="imagenet", input_tensor=image_input)  # Pretrained EfficientNet without final dense layers

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add a global average pooling layer
x = GlobalAveragePooling2D()(base_model.output)  # Convert 4D feature map to 2D vector (batch_size, 2048)

# Phylum input (one-hot or multi-class vector with 5 classes)
phylum_input = Input(shape=(5,), name="phylum_input")  # Input for phylum-level info

# Combine image and phylum features
combined = Concatenate()([x, phylum_input])  # Concatenate the two inputs: (batch_size, 2048 + 5)
combined = Dense(256, activation='relu')(combined)  # Fully connected layer
output = Dense(202, activation='softmax')(combined)  # Final classification layer (202 family classes)

# Define the model
model_en_no_proc_no_reg = Model(inputs=[image_input, phylum_input], outputs=output)

# Compile the model
model_en_no_proc_no_reg.compile(
    optimizer=optimizers.RMSprop(learning_rate=1e-4),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=metrics
)

# Print the model summary
# model.summary()

In [ ]:
# Initialize the experiment
experiment_en_no_proc_no_reg = Experiment(
    model=model_en_no_proc_no_reg,
    train_ds=train_ds_en_no_proc_no_reg,
    val_ds=val_ds_en_no_proc_no_reg,
    experiment_name="eff-net_with_phylum_no_proc_no_reg",
    resume=False,
    steps_per_epoch=263
)

# Run the experiment
history_en_no_proc_no_reg = experiment_en_no_proc_no_reg.run_experiment(callbacks=None, epochs=25)

Epoch 1/25
263/263 ━━━━━━━━━━━━━━━━━━━━ 1024s 907ms/step - accuracy: 0.0772 - auc: 0.6519 - f1_macro: 0.0165 - f1_weighted: 0.0529 - loss: 5.0453 - top5_accuracy: 0.1766 - val_accuracy: 0.2265 - val_auc: 0.8324 - val_f1_macro: 0.0506 - val_f1_weighted: 0.1326 - val_loss: 4.2145 - val_top5_accuracy: 0.4096
Epoch 2/25
263/263 ━━━━━━━━━━━━━━━━━━━━ 72s 276ms/step - accuracy: 0.2565 - auc: 0.8597 - f1_macro: 0.0669 - f1_weighted: 0.1603 - loss: 3.9490 - top5_accuracy: 0.4659 - val_accuracy: 0.3172 - val_auc: 0.9007 - val_f1_macro: 0.1207 - val_f1_weighted: 0.2208 - val_loss: 3.4347 - val_top5_accuracy: 0.5754
Epoch 3/25
263/263 ━━━━━━━━━━━━━━━━━━━━ 72s 274ms/step - accuracy: 0.3597 - auc: 0.9299 - f1_macro: 0.1572 - f1_weighted: 0.2688 - loss: 3.1741 - top5_accuracy: 0.6317 - val_accuracy: 0.3990 - val_auc: 0.9411 - val_f1_macro: 0.2059 - val_f1_weighted: 0.3155 - val_loss: 2.8685 - val_top5_accuracy: 0.6650
Epoch 4/25
263/263 ━━━━━━━━━━━━━━━━━━━━ 68s 260ms/step - accuracy: 0.4625 - auc: 0.

In [ ]:
# Load the experiment log
df = pd.read_csv('experiment_log.csv')

# Identify the latest experiment
max_id = df['id'].max()

# Filter the DataFrame to get the latest experiment
df_latest_experiment = df[df['id'] == max_id]

# Save the latest experiment log to a CSV file
# df_latest_experiment.to_csv('phylum_models_results/efficient_net_phylum_1_no_proc_no_reg_history.csv', index=False)

df_latest_experiment

,id,experiment_name,epoch,train_accuracy,val_accuracy,train_loss,val_loss,f1_train_macro,f1_val_macro,f1_train_weighted,f1_val_weighted,top5_train_accuracy,top5_val_accuracy,timestamp
504,20,eff-net_with_phylum_no_proc_no_reg,1,0.1415,0.2265,4.7762,4.2145,0.0308,0.0506,0.0914,0.1326,0.2750,0.4096,2025-04-27 20:01:01
505,20,eff-net_with_phylum_no_proc_no_reg,2,0.2755,0.3172,3.7638,3.4347,0.0881,0.1207,0.1822,0.2208,0.5070,0.5754,2025-04-27 20:02:14
506,20,eff-net_with_phylum_no_proc_no_reg,3,0.3760,0.3990,3.0499,2.8685,0.1809,0.2059,0.2898,0.3155,0.6495,0.6650,2025-04-27 20:03:25
507,20,eff-net_with_phylum_no_proc_no_reg,4,0.4742,0.4758,2.4993,2.4408,0.2981,0.2923,0.4051,0.4012,0.7456,0.7429,2025-04-27 20:04:34
508,20,eff-net_with_phylum_no_proc_no_reg,5,0.5573,0.5253,2.0807,2.1216,0.4090,0.3721,0.5022,0.4635,0.8152,0.7908,2025-04-27 20:05:42
509,20,eff-net_with_phylum_no_proc_no_reg,6,0.6116,0.5754,1.7624,1.8868,0.5034,0.4590,0.5714,0.5291,0.8655,0.8325,2025-04-27 20:06:50
510,20,eff-net_with_phylum_no_proc_no_reg,7,0.6631,0.5977,1.5212,1.7217,0.5887,0.4966,0.6350,0.5588,0.8941,0.8486,2025-04-27 20:08:01


### Base line 2 (without preprocessing and with regularization)

#### Set-up

In [ ]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input

In [ ]:
# Compile with metrics
verbose = 1
metrics = [
    CategoricalAccuracy(name="accuracy"),
    AUC(name="auc"),
    F1Score(average="macro", name="f1_macro"),
    F1Score(average="weighted", name="f1_weighted"),
    TopKCategoricalAccuracy(k=5, name="top5_accuracy")
]

In [ ]:
# Load datasets
train_ds_en_no_proc_reg, family_class_names, phylum_class_names = preprocess.load_img(
    train_df,
    minority_class=[],
    augment=None,
    oversampling=False,
    shuffle=True,
    preprocessing_function=preprocess_input,
    family_encoder=family_encoder,
    phylum_encoder=phylum_encoder,
)

val_ds_en_no_proc_reg, _, _ = preprocess.load_img(
    val_df,
    minority_class=[],
    augment=None,
    oversampling=False,
    shuffle=False,
    preprocessing_function=preprocess_input,
    family_encoder=family_encoder,
    phylum_encoder=phylum_encoder,
)

#### Run

In [ ]:
# Image input pipeline
image_input = Input(shape=(224, 224, 3), name="image_input")  # Input for RGB image
base_model = EfficientNetB0(include_top=False, weights="imagenet", input_tensor=image_input)  # Pretrained EfficientNet without final dense layers

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add a global average pooling layer
x = GlobalAveragePooling2D()(base_model.output)  # Convert 4D feature map to 2D vector (batch_size, 2048)

# Phylum input (one-hot or multi-class vector with 5 classes)
phylum_input = Input(shape=(5,), name="phylum_input")  # Input for phylum-level info

# Combine image and phylum features
combined = Concatenate()([x, phylum_input])  # Concatenate the two inputs: (batch_size, 2048 + 5)
combined = Dense(256, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(combined)  # regularization parameter
combined = layers.Dropout(0.5)(combined) # regularization layer
output = Dense(202, activation='softmax', kernel_regularizer=regularizers.l2(1e-4))(combined)  # Final classification layer (202 family classes)

# Define the model
model_en_no_proc_reg = Model(inputs=[image_input, phylum_input], outputs=output)

# Compile the model
model_en_no_proc_reg.compile(
    optimizer=optimizers.RMSprop(learning_rate=1e-4),
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.01), # regularization parameter
    metrics=metrics
)

# Print the model summary
# model.summary()

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
]

In [ ]:
# Initialize the experiment
experiment_en_no_proc_reg = Experiment(
    model=model_en_no_proc_reg,
    train_ds=train_ds_en_no_proc_reg,
    val_ds=val_ds_en_no_proc_reg,
    experiment_name="eff-net_with_phylum_no_proc_reg",
    resume=True,
    steps_per_epoch=263,
)

# Run the experiment
history_en_no_proc_reg = experiment_en_no_proc_reg.run_experiment(callbacks=callbacks, epochs=50)

Resuming training from epoch 14 (timestamp 20250427-203928)
Epoch 15/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 185s 342ms/step - accuracy: 0.6232 - auc: 0.9842 - f1_macro: 0.5272 - f1_weighted: 0.6002 - loss: 1.6938 - top5_accuracy: 0.8786 - val_accuracy: 0.6155 - val_auc: 0.9779 - val_f1_macro: 0.5193 - val_f1_weighted: 0.5788 - val_loss: 1.7588 - val_top5_accuracy: 0.8603
Epoch 16/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 72s 276ms/step - accuracy: 0.6369 - auc: 0.9842 - f1_macro: 0.5467 - f1_weighted: 0.6126 - loss: 1.6644 - top5_accuracy: 0.8827 - val_accuracy: 0.6188 - val_auc: 0.9788 - val_f1_macro: 0.5297 - val_f1_weighted: 0.5846 - val_loss: 1.7282 - val_top5_accuracy: 0.8614
Epoch 17/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 72s 273ms/step - accuracy: 0.6464 - auc: 0.9869 - f1_macro: 0.5688 - f1_weighted: 0.6272 - loss: 1.5897 - top5_accuracy: 0.8949 - val_accuracy: 0.6361 - val_auc: 0.9791 - val_f1_macro: 0.5556 - val_f1_weighted: 0.6063 - val_loss: 1.6974 - val_top5_accuracy: 0.8687
Epoch 18/50
263/263 ━

In [ ]:
# Load the experiment log
df = pd.read_csv('experiment_log.csv')

# Identify the latest experiment
max_id = df['id'].max()

# Filter the DataFrame to get the latest experiment
df_latest_experiment = df[df['id'] == max_id]

# Save the latest experiment log to a CSV file
df_latest_experiment.to_csv('phylum_models_results/efficient_net_phylum_2_no_proc_reg_history.csv', index=False)

df_latest_experiment

,id,experiment_name,epoch,train_accuracy,val_accuracy,train_loss,val_loss,f1_train_macro,f1_val_macro,f1_train_weighted,f1_val_weighted,top5_train_accuracy,top5_val_accuracy,timestamp
511,21,eff-net_with_phylum_no_proc_reg,1,0.0789,0.1903,5.0605,4.5924,0.0200,0.0412,0.0574,0.1143,0.1753,0.3495,2025-04-27 20:20:08
512,21,eff-net_with_phylum_no_proc_reg,2,0.1961,0.2571,4.3433,3.9565,0.0486,0.0687,0.1231,0.1605,0.3699,0.4786,2025-04-27 20:21:21
513,21,eff-net_with_phylum_no_proc_reg,3,0.2555,0.3155,3.8035,3.4960,0.0792,0.1089,0.1705,0.2156,0.4812,0.5626,2025-04-27 20:22:32
514,21,eff-net_with_phylum_no_proc_reg,4,0.3129,0.3728,3.3946,3.1410,0.1244,0.1692,0.2304,0.2828,0.5682,0.6266,2025-04-27 20:23:40
515,21,eff-net_with_phylum_no_proc_reg,5,0.3623,0.4174,3.0547,2.8612,0.1814,0.2212,0.2900,0.3355,0.6460,0.6650,2025-04-27 20:24:48
516,21,eff-net_with_phylum_no_proc_reg,6,0.4091,0.4541,2.8042,2.6321,0.2392,0.2566,0.3431,0.3744,0.6879,0.7117,2025-04-27 20:25:57
517,21,eff-net_with_phylum_no_proc_reg,7,0.4486,0.4919,2.5835,2.4393,0.2941,0.3044,0.3914,0.4214,0.7287,0.7518,2025-04-27 20:27:06
518,21,eff-net_with_phylum_no_proc_reg,8,0.4726,0.5198,2.4086,2.2862,0.3284,0.3476,0.4204,0.4541,0.7620,0.7730,2025-04-27 20:32:30
519,21,eff-net_with_phylum_no_proc_reg,9,0.5012,0.5392,2.2529,2.1568,0.3662,0.3878,0.4536,0.4796,0.7896,0.7952,2025-04-27 20:33:41
520,21,eff-net_with_phylum_no_proc_reg,10,0.5250,0.5515,2.1254,2.0574,0.4099,0.4077,0.4848,0.4958,0.8073,0.8114,2025-04-27 20:34:51


### Set-up

In [ ]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input

In [ ]:
verbose = 1
metrics = [
    CategoricalAccuracy(name="accuracy"),
    AUC(name="auc"),
    F1Score(average="macro", name="f1_macro"),
    F1Score(average="weighted", name="f1_weighted"),
    TopKCategoricalAccuracy(k=5, name="top5_accuracy")
]

initial_lr = 1e-4
final_lr = 1e-5
n_epochs = 50
my_scheduler_fn = lr_scheduler(initial_lr, final_lr, n_epochs)
lr_callback = LearningRateScheduler(my_scheduler_fn)

In [ ]:
# Load datasets
train_ds_en_pre_ft, family_class_names, phylum_class_names = preprocess.load_img(
    train_df,
    minority_class=minority_class,
    augment='mixup',
    oversampling=True,
    shuffle=True,
    preprocessing_function=preprocess_input,
    family_encoder=family_encoder,
    phylum_encoder=phylum_encoder,
)

val_ds_en_pre_ft, _, _ = preprocess.load_img(
    val_df,
    minority_class=[],
    augment=None,
    oversampling=False,
    shuffle=False,
    preprocessing_function=preprocess_input,
    family_encoder=family_encoder,
    phylum_encoder=phylum_encoder,
)

### Head Train

In [ ]:
# Image input pipeline
image_input = Input(shape=(224, 224, 3), name="image_input")  # Input for RGB image
base_model = EfficientNetB0(include_top=False, weights="imagenet", input_tensor=image_input)  # Pretrained EfficientNet without final dense layers

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add a global average pooling layer
x = GlobalAveragePooling2D()(base_model.output)  # Convert 4D feature map to 2D vector (batch_size, 2048)

# Phylum input (one-hot or multi-class vector with 5 classes)
phylum_input = Input(shape=(5,), name="phylum_input")  # Input for phylum-level info

# Combine image and phylum features
combined = Concatenate()([x, phylum_input])  # Concatenate the two inputs: (batch_size, 2048 + 5)
combined = Dense(256, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(combined)  # regularization parameter
combined = layers.Dropout(0.5)(combined) # regularization layer
output = Dense(202, activation='softmax', kernel_regularizer=regularizers.l2(1e-4))(combined)  # Final classification layer (202 family classes)

# Define the model
model_en_pre_ft = Model(inputs=[image_input, phylum_input], outputs=output)

# Compile the model
model_en_pre_ft.compile(
    optimizer=optimizers.RMSprop(learning_rate=1e-4),
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.01), # regularization parameter
    metrics=metrics
)

# Print the model summary
# model.summary()

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    lr_callback
]

In [ ]:
# Initialize the experiment
experiment_en_pre_ft = Experiment(
    model=model_en_pre_ft,
    train_ds=train_ds_en_pre_ft,
    val_ds=val_ds_en_pre_ft,
    experiment_name="eff-net_with_phylum_pre-ft",
    resume=True,
    steps_per_epoch=350,
)

# Run the experiment
history_en_pre_ft = experiment_en_pre_ft.run_experiment(callbacks=callbacks, epochs=50)

Resuming training from epoch 4 (timestamp 20250430-112706)
Epoch 5/50
350/350 ━━━━━━━━━━━━━━━━━━━━ 198s 268ms/step - accuracy: 0.3537 - auc: 0.9448 - f1_macro: 0.2559 - f1_weighted: 0.3031 - loss: 3.0786 - top5_accuracy: 0.6576 - val_accuracy: 0.4485 - val_auc: 0.9671 - val_f1_macro: 0.3228 - val_f1_weighted: 0.3807 - val_loss: 2.6730 - val_top5_accuracy: 0.7557 - learning_rate: 7.9433e-05
Epoch 6/50
350/350 ━━━━━━━━━━━━━━━━━━━━ 73s 208ms/step - accuracy: 0.3979 - auc: 0.9563 - f1_macro: 0.3128 - f1_weighted: 0.3524 - loss: 2.8115 - top5_accuracy: 0.7146 - val_accuracy: 0.4914 - val_auc: 0.9733 - val_f1_macro: 0.3877 - val_f1_weighted: 0.4350 - val_loss: 2.4497 - val_top5_accuracy: 0.7908 - learning_rate: 7.5858e-05
Epoch 7/50
350/350 ━━━━━━━━━━━━━━━━━━━━ 72s 206ms/step - accuracy: 0.4438 - auc: 0.9635 - f1_macro: 0.3739 - f1_weighted: 0.4098 - loss: 2.5650 - top5_accuracy: 0.7551 - val_accuracy: 0.5281 - val_auc: 0.9752 - val_f1_macro: 0.4346 - val_f1_weighted: 0.4791 - val_loss: 2.27

In [ ]:
# Load the experiment log
df = pd.read_csv('experiment_log.csv')

# Identify the latest experiment
max_id = df['id'].max()

# Filter the DataFrame to get the latest experiment
df_latest_experiment = df[df['id'] == max_id]

# Save the latest experiment log to a CSV file
df_latest_experiment.to_csv('phylum_models_results/efficient_net_phylum_3_pre_ft_history.csv', index=False)

df_latest_experiment

,id,experiment_name,epoch,train_accuracy,val_accuracy,train_loss,val_loss,f1_train_macro,f1_val_macro,f1_train_weighted,f1_val_weighted,top5_train_accuracy,top5_val_accuracy,timestamp
598,26,eff-net_with_phylum_pre-ft,1,0.0725,0.1959,5.1128,4.5651,0.0275,0.0564,0.0501,0.1194,0.1632,0.4101,2025-04-30 11:23:28
599,26,eff-net_with_phylum_pre-ft,2,0.1701,0.2710,4.4134,3.8504,0.0708,0.1148,0.1083,0.1891,0.3539,0.5287,2025-04-30 11:24:40
600,26,eff-net_with_phylum_pre-ft,3,0.2371,0.3417,3.8328,3.3416,0.1307,0.1771,0.1699,0.2553,0.4883,0.6238,2025-04-30 11:25:51
601,26,eff-net_with_phylum_pre-ft,4,0.2946,0.3912,3.4055,2.9646,0.2007,0.2373,0.2345,0.3083,0.5884,0.7034,2025-04-30 11:27:06
602,26,eff-net_with_phylum_pre-ft,5,0.3517,0.4485,3.0546,2.6730,0.2798,0.3228,0.3045,0.3807,0.6604,0.7557,2025-04-30 11:32:06
603,26,eff-net_with_phylum_pre-ft,6,0.4068,0.4914,2.7712,2.4497,0.3460,0.3877,0.3684,0.4350,0.7201,0.7908,2025-04-30 11:33:19
604,26,eff-net_with_phylum_pre-ft,7,0.4472,0.5281,2.5439,2.2779,0.4030,0.4346,0.4164,0.4791,0.7565,0.8119,2025-04-30 11:34:31
605,26,eff-net_with_phylum_pre-ft,8,0.4817,0.5570,2.3788,2.1500,0.4384,0.4795,0.4532,0.5164,0.7805,0.8303,2025-04-30 11:35:45


### Fine-tune

References: https://www.tensorflow.org/tutorials/keras/keras_tuner?hl=pt-br

"When fine-tuning a pre-trained model that contains BatchNormalization layers, it is usually a good idea to keep them frozen (i.e., set layer.trainable = False), to avoid corrupting the running statistics that the layers have learned."

In [19]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input

In [20]:
model_en_ft = load_model("model_eff-net_with_phylum_pre-ft_20250429-100801.keras", compile=False)

In [21]:
verbose = 1
metrics = [
    CategoricalAccuracy(name="accuracy"),
    AUC(name="auc"),
    F1Score(average="macro", name="f1_macro"),
    F1Score(average="weighted", name="f1_weighted"),
    TopKCategoricalAccuracy(k=5, name="top5_accuracy")
]

initial_lr = 1e-5
final_lr = 1e-6
n_epochs = 100
my_scheduler_fn = lr_scheduler(initial_lr, final_lr, n_epochs)
lr_callback = LearningRateScheduler(my_scheduler_fn)

In [22]:
progressive_unfreeze = ProgressiveUnfreeze(model_en_ft) # class define in classes file

model_en_ft.compile(
    optimizer=optimizers.RMSprop(learning_rate=1e-5),
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.01),
    metrics=metrics
)

In [23]:
# Load datasets
train_ds_en_pre_ft, family_class_names, phylum_class_names = preprocess.load_img(
    train_df,
    minority_class=minority_class,
    augment='mixup',
    oversampling=True,
    shuffle=True,
    preprocessing_function=preprocess_input,
    family_encoder=family_encoder,
    phylum_encoder=phylum_encoder,
)

val_ds_en_pre_ft, _, _ = preprocess.load_img(
    val_df,
    minority_class=[],
    augment=None,
    oversampling=False,
    shuffle=False,
    preprocessing_function=preprocess_input,
    family_encoder=family_encoder,
    phylum_encoder=phylum_encoder,
)

In [ ]:
# Initialize the experiment
experiment_en_ft = Experiment(
    model=model_en_ft,
    train_ds=train_ds_en_pre_ft,
    val_ds=val_ds_en_pre_ft,
    experiment_name="eff-net_with_phylum_ft",
    resume=True,
    steps_per_epoch=350,
)

# Run the experiment
history_ft_en = experiment_en_ft.run_experiment(
    epochs=100,
    callbacks=[progressive_unfreeze, lr_callback]
)

Resuming training from epoch 52 (timestamp 20250430-231931)
Epoch 53/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step - accuracy: 0.7012 - auc: 0.9895 - f1_macro: 0.6681 - f1_weighted: 0.6954 - loss: 1.4434 - top5_accuracy: 0.9188Epoch 52: Layer now unfrozen 243 (dense_1)
350/350 ━━━━━━━━━━━━━━━━━━━━ 190s 257ms/step - accuracy: 0.7011 - auc: 0.9895 - f1_macro: 0.6681 - f1_weighted: 0.6954 - loss: 1.4435 - top5_accuracy: 0.9188 - val_accuracy: 0.6667 - val_auc: 0.9818 - val_f1_macro: 0.6219 - val_f1_weighted: 0.6474 - val_loss: 1.6243 - val_top5_accuracy: 0.8815 - learning_rate: 2.9512e-06
Epoch 54/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 70s 201ms/step - accuracy: 0.6903 - auc: 0.9901 - f1_macro: 0.6538 - f1_weighted: 0.6839 - loss: 1.4816 - top5_accuracy: 0.9121 - val_accuracy: 0.6661 - val_auc: 0.9818 - val_f1_macro: 0.6208 - val_f1_weighted: 0.6467 - val_loss: 1.6240 - val_top5_accuracy: 0.8815 - learning_rate: 2.8840e-06
Epoch 55/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step - accuracy: 0.6

In [14]:
# Load the experiment log
df = pd.read_csv('experiment_log.csv')

# Identify the latest experiment
max_id = df['id'].max()

# Filter the DataFrame to get the latest experiment
df_latest_experiment = df[df['id'] == max_id]

# Save the latest experiment log to a CSV file
df_latest_experiment.to_csv('phylum_models_results/efficient_net_phylum_4_ft_history.csv', index=False)

df_latest_experiment

,id,experiment_name,epoch,train_accuracy,val_accuracy,train_loss,val_loss,f1_train_macro,f1_val_macro,f1_train_weighted,f1_val_weighted,top5_train_accuracy,top5_val_accuracy,timestamp
606,27,eff-net_with_phylum_ft,1,0.6519,0.6477,1.6082,1.6892,0.6426,0.5973,0.6431,0.6258,0.8955,0.8726,2025-04-30 11:49:35
607,27,eff-net_with_phylum_ft,2,0.6519,0.6489,1.6174,1.6862,0.6461,0.5999,0.6449,0.6270,0.8950,0.8715,2025-04-30 11:50:47
608,27,eff-net_with_phylum_ft,3,0.6521,0.6483,1.6055,1.6833,0.6452,0.5999,0.6455,0.6266,0.8950,0.8737,2025-04-30 11:51:58
609,27,eff-net_with_phylum_ft,4,0.6571,0.6483,1.6050,1.6806,0.6515,0.5990,0.6501,0.6264,0.8936,0.8726,2025-04-30 11:53:11
610,27,eff-net_with_phylum_ft,5,0.6581,0.6494,1.5960,1.6785,0.6517,0.6004,0.6510,0.6275,0.8942,0.8726,2025-04-30 11:59:50
611,27,eff-net_with_phylum_ft,6,0.6549,0.6494,1.6015,1.6763,0.6460,0.6002,0.6478,0.6275,0.8952,0.8731,2025-04-30 12:01:01
612,27,eff-net_with_phylum_ft,7,0.6563,0.6489,1.5920,1.6737,0.6497,0.6001,0.6488,0.6271,0.8962,0.8737,2025-04-30 12:02:13
613,27,eff-net_with_phylum_ft,8,0.6594,0.6505,1.5886,1.6714,0.6525,0.6011,0.6531,0.6287,0.8974,0.8726,2025-04-30 12:03:26
614,27,eff-net_with_phylum_ft,9,0.6623,0.6522,1.5750,1.6693,0.6550,0.6053,0.6542,0.6319,0.9008,0.8726,2025-04-30 15:42:51
615,27,eff-net_with_phylum_ft,10,0.6611,0.6522,1.5942,1.6676,0.6564,0.6055,0.6545,0.6319,0.8990,0.8737,2025-04-30 15:44:02


### Metrics

In [ ]:
# get_metric(val_ds_en_pre_ft, "efficient_net_finetuned_final.keras")
# get_metric(val_ds_en_pre_ft,"efficient_net_pre_finetuning_with_label_smoothing_batch_sized_corrected_launching_trying_lr_scheduler.keras")
# get_metric(val_ds_en_no_proc_reg, "model_Efficient_net_baseline2_20250423-115426.keras")
# get_metric(val_ds_en_no_proc_no_reg, "model_Efficient_net_baseline1_20250423-104546.keras")
# # custom function to include the metrics per class, as well as the precision and recall for error analysis purposes


Evaluating model: efficient_net_finetuned_final.keras


/Users/margaridabravocardoso/venvs/deep_learning_venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 34 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
2025-04-24 11:36:30.785988: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/Users/margaridabravocardoso/venvs/deep_learning_venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/margaridabravocardoso/venvs/deep_learning_venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defi

              precision    recall  f1-score   support

           0     0.7391    0.9444    0.8293        18
           1     0.6429    0.6923    0.6667        13
           2     0.6579    0.8065    0.7246        31
           3     0.7143    0.5556    0.6250         9
           4     0.6364    0.7778    0.7000        18
           5     0.6000    0.6000    0.6000         5
           6     0.0000    0.0000    0.0000         4
           7     0.6667    0.4000    0.5000         5
           8     1.0000    0.1111    0.2000         9
           9     1.0000    0.3333    0.5000         9
          10     0.6364    0.7778    0.7000        27
          11     0.7500    0.6667    0.7059         9
          12     1.0000    0.5556    0.7143         9
          13     1.0000    1.0000    1.0000        22
          14     0.7143    0.5556    0.6250         9
          15     1.0000    0.5000    0.6667         4
          16     0.5909    0.5909    0.5909        22
          17     1.0000    

2025-04-24 11:37:04.922697: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/Users/margaridabravocardoso/venvs/deep_learning_venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/margaridabravocardoso/venvs/deep_learning_venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/margaridabravocardoso/venvs/deep_learning_venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: Und

              precision    recall  f1-score   support

           0     0.6957    0.8889    0.7805        18
           1     0.6429    0.6923    0.6667        13
           2     0.6579    0.8065    0.7246        31
           3     0.7143    0.5556    0.6250         9
           4     0.6364    0.7778    0.7000        18
           5     0.5000    0.4000    0.4444         5
           6     0.0000    0.0000    0.0000         4
           7     0.7500    0.6000    0.6667         5
           8     1.0000    0.1111    0.2000         9
           9     1.0000    0.2222    0.3636         9
          10     0.6250    0.7407    0.6780        27
          11     0.7500    0.6667    0.7059         9
          12     1.0000    0.5556    0.7143         9
          13     1.0000    1.0000    1.0000        22
          14     0.7143    0.5556    0.6250         9
          15     1.0000    0.5000    0.6667         4
          16     0.6087    0.6364    0.6222        22
          17     1.0000    

2025-04-24 11:37:26.793685: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/Users/margaridabravocardoso/venvs/deep_learning_venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/margaridabravocardoso/venvs/deep_learning_venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/margaridabravocardoso/venvs/deep_learning_venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: Und

              precision    recall  f1-score   support

           0     0.6296    0.9444    0.7556        18
           1     0.5556    0.7692    0.6452        13
           2     0.6341    0.8387    0.7222        31
           3     0.8000    0.4444    0.5714         9
           4     0.5769    0.8333    0.6818        18
           5     1.0000    0.2000    0.3333         5
           6     0.0000    0.0000    0.0000         4
           7     1.0000    0.6000    0.7500         5
           8     1.0000    0.1111    0.2000         9
           9     1.0000    0.3333    0.5000         9
          10     0.5676    0.7778    0.6562        27
          11     0.4000    0.6667    0.5000         9
          12     1.0000    0.5556    0.7143         9
          13     1.0000    0.9545    0.9767        22
          14     0.5556    0.5556    0.5556         9
          15     1.0000    0.2500    0.4000         4
          16     0.5926    0.7273    0.6531        22
          17     1.0000    

2025-04-24 11:37:50.245592: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/Users/margaridabravocardoso/venvs/deep_learning_venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/margaridabravocardoso/venvs/deep_learning_venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/margaridabravocardoso/venvs/deep_learning_venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: Und